In [1]:
import pandas as pd
import numpy as np
import os
import re
import collections
import unidecode
import nltk
from nltk.corpus import stopwords
import itertools 
from nltk.tokenize import word_tokenize
from string import punctuation

In [2]:
pd.set_option('display.max_colwidth', 1200)

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt

## Load data train

In [160]:
train = pd.read_csv("data/data_train/train_data_beto_10_NOamb_lfnorm_medline.csv")

## Study data train

In [79]:
df_pred = pd.read_csv("data/data_predict/df_train_10_pred_amb_lfnorm_medline.csv")

In [80]:
df_pred.head()

,Unnamed: 0,short_form,context,long_form,label,sentences,Prediction_train1,Prediction_train2,Prediction_train3
0,0,GPT,"de 68 . Los parámetros bioquímicos en sangre eran normales salvo la GOT 87 , < start > GPT < end > 51 , gamma-GT 67 y las proteínas totales que eran de 5,8 g/dl con albúmina",glutamic pyruvic transaminar,1,"[CLS] glutamic pyruvic transaminar [SEP] de 68 . Los parámetros bioquímicos en sangre eran normales salvo la GOT 87 , < start > GPT < end > 51 , gamma-GT 67 y las proteínas totales que eran de 5,8 g/dl con albúmina [SEP]",0.0,0.0,0.0
1,1,GPT,"de 68 . Los parámetros bioquímicos en sangre eran normales salvo la GOT 87 , < start > GPT < end > 51 , gamma-GT 67 y las proteínas totales que eran de 5,8 g/dl con albúmina",glutamate pyruvatar transaminar,0,"[CLS] glutamate pyruvatar transaminar [SEP] de 68 . Los parámetros bioquímicos en sangre eran normales salvo la GOT 87 , < start > GPT < end > 51 , gamma-GT 67 y las proteínas totales que eran de 5,8 g/dl con albúmina [SEP]",0.0,0.0,0.0
2,2,C,"A la exploración física destacaba la delgadez ( IMC 17 ) , temperatura 37 º < start > C < end > tensión arterial 105/60 , se encontraba consciente y orientada , con exploración neurológica completa normal",centimetro,1,"[CLS] centimetro [SEP] A la exploración física destacaba la delgadez ( IMC 17 ) , temperatura 37 º < start > C < end > tensión arterial 105/60 , se encontraba consciente y orientada , con exploración neurológica completa normal [SEP]",0.0,0.0,0.0
3,3,C,"A la exploración física destacaba la delgadez ( IMC 17 ) , temperatura 37 º < start > C < end > tensión arterial 105/60 , se encontraba consciente y orientada , con exploración neurológica completa normal",centigrado,0,"[CLS] centigrado [SEP] A la exploración física destacaba la delgadez ( IMC 17 ) , temperatura 37 º < start > C < end > tensión arterial 105/60 , se encontraba consciente y orientada , con exploración neurológica completa normal [SEP]",0.0,0.0,0.0
4,4,C,"A la exploración física destacaba la delgadez ( IMC 17 ) , temperatura 37 º < start > C < end > tensión arterial 105/60 , se encontraba consciente y orientada , con exploración neurológica completa normal",cysteine,0,"[CLS] cysteine [SEP] A la exploración física destacaba la delgadez ( IMC 17 ) , temperatura 37 º < start > C < end > tensión arterial 105/60 , se encontraba consciente y orientada , con exploración neurológica completa normal [SEP]",0.0,0.0,0.0


In [36]:
df_pred.shape

(2232, 9)

Veamos cuantos acrónimos son ambiguos

In [37]:
amb = df_pred.groupby('short_form')['long_form'].nunique().reset_index().sort_values('long_form', ascending = False)
print(amb.shape)
amb[amb['long_form'] == 1].shape

(54, 2)


(0, 2)

Filtramos para aquellas no ha acertado

In [81]:
wrong = df_pred[((df_pred['label'] == 0) & (df_pred['Prediction_train1'] == 1.0)) | (df_pred['label'] == 1) & (df_pred['Prediction_train1'] == 0.0)]

In [82]:
wrong.shape

(109, 9)

In [83]:
del wrong['Unnamed: 0']

In [84]:
wrong.sort_values(['short_form'])

,short_form,context,long_form,label,sentences,Prediction_train1,Prediction_train2,Prediction_train3
194,ALT,"predominio colestásico ( fosfatasa alcalina y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ) , sin insuficiencia hepática .",alanine transferar,0,"[CLS] alanine transferar [SEP] predominio colestásico ( fosfatasa alcalina y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ) , sin insuficiencia hepática . [SEP]",1.0,0.0,0.0
191,ALT,perfil hepático presentando una importante elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con importante aumento de la LDH,alanine transferar,0,[CLS] alanine transferar [SEP] perfil hepático presentando una importante elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con importante aumento de la LDH [SEP],1.0,0.0,0.0
190,ALT,perfil hepático presentando una importante elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con importante aumento de la LDH,alanine transaminar,1,[CLS] alanine transaminar [SEP] perfil hepático presentando una importante elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con importante aumento de la LDH [SEP],0.0,0.0,0.0
188,AST,"( fosfatasa alcalina y GGT más de 10 veces el valor normal y ALT y < start > AST < end > menos de 3 veces el valor normal ) , sin insuficiencia hepática . A las",aspartate and alanine aminotransferar,0,"[CLS] aspartate and alanine aminotransferar [SEP] ( fosfatasa alcalina y GGT más de 10 veces el valor normal y ALT y < start > AST < end > menos de 3 veces el valor normal ) , sin insuficiencia hepática . A las [SEP]",1.0,0.0,0.0
184,AST,presentando una importante elevación de transaminasas en el rango de hepatitis aguda ( ALT y < start > AST < end > mayores de 20 veces el valor normal con importante aumento de la LDH ) y,aspartato aminotransferasar,1,[CLS] aspartato aminotransferasar [SEP] presentando una importante elevación de transaminasas en el rango de hepatitis aguda ( ALT y < start > AST < end > mayores de 20 veces el valor normal con importante aumento de la LDH ) y [SEP],0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
64,mm,"nueva dilatación del píloro , en este caso forzada , con un balón de 35 < start > mm < end > ( Rigiflex® ; Microvasive , Boston Scientific , EE . UU . ) , previo",milimetro,1,"[CLS] milimetro [SEP] nueva dilatación del píloro , en este caso forzada , con un balón de 35 < start > mm < end > ( Rigiflex® ; Microvasive , Boston Scientific , EE . UU . ) , previo [SEP]",0.0,0.0,0.0
66,mm,"píloro marcadamente estenosado . Se llevó a cabo una dilatación pilórica con balón de 18 < start > mm < end > ( CRE™ Wireguided Balloon Dilator ; Boston Scientific Corporation ) , pasando a continuación un",milimetro,1,"[CLS] milimetro [SEP] píloro marcadamente estenosado . Se llevó a cabo una dilatación pilórica con balón de 18 < start > mm < end > ( CRE™ Wireguided Balloon Dilator ; Boston Scientific Corporation ) , pasando a continuación un [SEP]",0.0,0.0,0.0
67,mm,"píloro marcadamente estenosado . Se llevó a cabo una dilatación pilórica con balón de 18 < start > mm < end > ( CRE™ Wireguided Balloon Dilator ; Boston Scientific Corporation ) , pasando a continuación un",milimetro mercurio,0,"[CLS] milimetro mercurio [SEP] píloro marcadamente estenosado . Se llevó a cabo una dilatación pilórica con balón de 18 < start > mm < end > ( CRE™ Wireguided Balloon Dilator ; Boston Scientific Corporation ) , pasando a continuación un [SEP]",1.0,0.0,0.0
68,mm,"Balloon Dilator ; Boston Scientific Corporation ) , pasando a continuación un gastroscopio de 10 < start > mm < end > de forma fácil por dicho canal . Sin embargo , después de unos días ,",milime

In [85]:
wrong.groupby('short_form').size().reset_index().sort_values(0, ascending = False)

,short_form,0
12,cm,42
2,C,12
6,H2O,11
8,PCR,7
15,mm,7
13,h,5
7,L,5
1,AST,4
14,kg,3
0,ALT,3


Estudiemos cada acrónimo que da fallo

In [51]:
wrong[wrong['short_form'] == 'ALT']

,short_form,context,long_form,label,sentences,Prediction_train1,Prediction_train2,Prediction_train3
190,ALT,elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con,alanine transaminar,1,[CLS] alanine transaminar [SEP] elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con [SEP],0.0,0.0,0.0
191,ALT,elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con,alanine transferar,0,[CLS] alanine transferar [SEP] elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con [SEP],1.0,0.0,0.0
194,ALT,y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ),alanine transferar,0,[CLS] alanine transferar [SEP] y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ) [SEP],1.0,0.0,0.0


In [50]:
df_pred[(df_pred['short_form'] == 'ALT')]['long_form'].unique()

array(['alanine transaminar', 'alanine transferar',
       'alanina aminotransferaso'], dtype=object)

In [49]:
df_pred[(df_pred['short_form'] == 'ALT') & (df_pred['label'] == 1)]

,Unnamed: 0,short_form,context,long_form,label,sentences,Prediction_train1,Prediction_train2,Prediction_train3
190,190,ALT,elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con,alanine transaminar,1,[CLS] alanine transaminar [SEP] elevación de transaminasas en el rango de hepatitis aguda ( < start > ALT < end > y AST mayores de 20 veces el valor normal con [SEP],0.0,0.0,0.0
193,193,ALT,y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ),alanine transaminar,1,[CLS] alanine transaminar [SEP] y GGT más de 10 veces el valor normal y < start > ALT < end > y AST menos de 3 veces el valor normal ) [SEP],1.0,0.0,0.0
459,459,ALT,"mg/dl , bilirrubina 1,1 mg/dl , AST 141 UI/l , < start > ALT < end > 38 UI/l , Fosfatasa alcalina 245 UI/l , LDH 1650",alanine transferar,1,"[CLS] alanine transferar [SEP] mg/dl , bilirrubina 1,1 mg/dl , AST 141 UI/l , < start > ALT < end > 38 UI/l , Fosfatasa alcalina 245 UI/l , LDH 1650 [SEP]",NaN,NaN,NaN
717,717,ALT,valores de las transaminasas ( AST : 304 U/L ; < start > ALT < end > 227 U/L ) y de LDH ( 568 U/L ),alanine transferar,1,[CLS] alanine transferar [SEP] valores de las transaminasas ( AST : 304 U/L ; < start > ALT < end > 227 U/L ) y de LDH ( 568 U/L ) [SEP],NaN,NaN,NaN
720,720,ALT,importante elevación de transaminasas ( AST : 5.550 U/L ; < start > ALT < end > 3.826 U/L ) y de LDH ( 10.375 U/L ),alanine transferar,1,[CLS] alanine transferar [SEP] importante elevación de transaminasas ( AST : 5.550 U/L ; < start > ALT < end > 3.826 U/L ) y de LDH ( 10.375 U/L ) [SEP],NaN,NaN,NaN
734,734,ALT,valores de las transaminasas ( AST : 304 U/L ; < start > ALT < end > 227 U/L ) y de LDH ( 568 U/L ),alanina aminotransferaso,1,[CLS] alanina aminotransferaso [SEP] valores de las transaminasas ( AST : 304 U/L ; < start > ALT < end > 227 U/L ) y de LDH ( 568 U/L ) [SEP],NaN,NaN,NaN
737,737,ALT,importante elevación de transaminasas ( AST : 5.550 U/L ; < start > ALT < end > 3.826 U/L ) y de LDH ( 10.375 U/L ),alanina aminotransferaso,1,[CLS] alanina aminotransferaso [SEP] importante elevación de transaminasas ( AST : 5.550 U/L ; < start > ALT < end > 3.826 U/L ) y de LDH ( 10.375 U/L ) [SEP],NaN,NaN,NaN
1058,1058,ALT,"fracción directa de 5,7 mg/dl , AST 312 U/l , < start > ALT < end > 106 U/l , GGT 416 U/l . Con el diagnóstico",alanine transferar,1,"[CLS] alanine transferar [SEP] fracción directa de 5,7 mg/dl , AST 312 U/l , < start > ALT < end > 106 U/l , GGT 416 U/l . Con el diagnóstico [SEP]",NaN,NaN,NaN
1076,1076,ALT,"Globulinas 3.2 , Fosfatasa alcalina 255 , GGT 22 , < start > ALT < end > 49 , AST 65 , Glicemia 148 y Creatinina 0.55",alanine transferar,1,"[CLS] alanine transferar [SEP] Globulinas 3.2 , Fosfatasa alcalina 255 , GGT 22 , < start > ALT < end > 49 , AST 65 , Glicemia 148 y Creatinina 0.55 [SEP]",NaN,NaN,NaN
1087,1087,ALT,91mg/dl ; bilirrubina total 21mg/dl ; AST 164 U/l ; < start > ALT < end > 103 U/l ; GGT 152 UI/dl . Marcadores virales hepáticos,alanina aminotransferaso,1,[CLS] alanina aminotransferaso [SEP] 91mg/dl ; bilirrubina total 21mg/dl ; AST 164 U/l ; < start > ALT < end > 103 U/l ; GGT 152 UI/dl . Marcadores virales hepáticos [SEP],NaN,NaN,NaN


#### Stuydy LF with less rows

In [69]:
train = pd.read_csv("data/data_train/train_data_beto_10_amb_lfnorm.csv")

In [70]:
train.head()

,Unnamed: 0,short_form,context,long_form,label
0,0,GPT,"bioquímicos en sangre eran normales salvo la GOT 87 , < start > GPT < end > 51 , gamma-GT 67 y las proteínas totales que eran",glutamic pyruvic transaminar,1
1,1,GPT,"bioquímicos en sangre eran normales salvo la GOT 87 , < start > GPT < end > 51 , gamma-GT 67 y las proteínas totales que eran",glutamate pyruvatar transaminar,0
2,2,C,"la delgadez ( IMC 17 ) , temperatura 37 º < start > C < end > tensión arterial 105/60 , se encontraba consciente y orientada ,",centimetro,1
3,3,C,"la delgadez ( IMC 17 ) , temperatura 37 º < start > C < end > tensión arterial 105/60 , se encontraba consciente y orientada ,",centigrado,0
4,4,C,"la delgadez ( IMC 17 ) , temperatura 37 º < start > C < end > tensión arterial 105/60 , se encontraba consciente y orientada ,",cysteine,0


In [72]:
train.groupby('long_form').size().reset_index().sort_values(0)

,long_form,0
43,cumulo diferenciacion 30,2
112,tomografia computarizado,2
20,beta,2
19,basofilo,2
113,tubular,2
...,...,...
81,microgramo,110
83,milimetro,114
84,milimetro mercurio,118
25,centimetro agua,227


## Study data test

In [161]:
test = pd.read_csv("data/data_predict/df_test_10_pred_NOamb_lfnorm_medline.csv")

In [162]:
del test['Unnamed: 0']
del test['Unnamed: 0.1']

In [163]:
test.head()

,short_form,context,long_form,label,sentences,Prediction
0,PAI,retiró el drenaje percutáneo . Con el diagnóstico de probable < start > PAI < end > el paciente fue dado de alta y seguido de forma,pancreatitis autoinmunir,1,[CLS] pancreatitis autoinmunir [SEP] retiró el drenaje percutáneo . Con el diagnóstico de probable < start > PAI < end > el paciente fue dado de alta y seguido de forma [SEP],1
1,SLA,"AMA , AMA-2 , ANCA , LKM1 , SMA , < start > SLA < end > y F-actina ) y serologías para virus y bacterias (",antisoluble liver antiger,1,"[CLS] antisoluble liver antiger [SEP] AMA , AMA-2 , ANCA , LKM1 , SMA , < start > SLA < end > y F-actina ) y serologías para virus y bacterias ( [SEP]",1
2,SMA,"ANA , AMA , AMA-2 , ANCA , LKM1 , < start > SMA < end > SLA y F-actina ) y serologías para virus y bacterias",smooth muscle actin,1,"[CLS] smooth muscle actin [SEP] ANA , AMA , AMA-2 , ANCA , LKM1 , < start > SMA < end > SLA y F-actina ) y serologías para virus y bacterias [SEP]",1
3,LKM1,"autoanticuerpos ( ANA , AMA , AMA-2 , ANCA , < start > LKM1 < end > SMA , SLA y F-actina ) y serologías para virus",liverkidney microsomal type 1,1,"[CLS] liverkidney microsomal type 1 [SEP] autoanticuerpos ( ANA , AMA , AMA-2 , ANCA , < start > LKM1 < end > SMA , SLA y F-actina ) y serologías para virus [SEP]",1
4,ANCA,". Los autoanticuerpos ( ANA , AMA , AMA-2 , < start > ANCA < end > LKM1 , SMA , SLA y F-actina ) y serologías",antineutrophil cytoplasmic antibodie,1,"[CLS] antineutrophil cytoplasmic antibodie [SEP] . Los autoanticuerpos ( ANA , AMA , AMA-2 , < start > ANCA < end > LKM1 , SMA , SLA y F-actina ) y serologías [SEP]",1


In [165]:
test.shape

(4419, 6)

Filtramos para aquellas no ha acertado

In [166]:
wrong_test = test[((test['label'] == 0) & (test['Prediction'] == 1)) | (test['label'] == 1) & (test['Prediction'] == 0)]

In [167]:
wrong_test.shape

(915, 6)

In [95]:
wrong_test.sort_values(['short_form']).head()

,short_form,context,long_form,label,sentences,Prediction
976,A,realizándose posteriormente una electroforesis que identificó la mutación 1691 G- < start > A < end > del factor V y 20210 G-A del factor II .,adenina,1,[CLS] adenina [SEP] realizándose posteriormente una electroforesis que identificó la mutación 1691 G- < start > A < end > del factor V y 20210 G-A del factor II . [SEP],0
883,A,dominante ( ATR-AD ) . Dicha mutación ( c.1766G > < start > A < end > fig . 2A ) supone la sustitución de un aminoácido,adeniner,1,[CLS] adeniner [SEP] dominante ( ATR-AD ) . Dicha mutación ( c.1766G > < start > A < end > fig . 2A ) supone la sustitución de un aminoácido [SEP],0
974,A,la mutación 1691 G-A del factor V y 20210 G- < start > A < end > del factor II . El resto del estudio incluyó hemograma,adenina,1,[CLS] adenina [SEP] la mutación 1691 G-A del factor V y 20210 G- < start > A < end > del factor II . El resto del estudio incluyó hemograma [SEP],0
1525,AL,( EVA < 2 ) con la nueva concentración de < start > AL < end > El catéter epidural se retiró en el quinto día del,amiloidosis primario,1,[CLS] amiloidosis primario [SEP] ( EVA < 2 ) con la nueva concentración de < start > AL < end > El catéter epidural se retiró en el quinto día del [SEP],0
1536,AL,( EVA < 2 ) con la nueva concentración de < start > AL < end > El catéter epidural se retiró en el quinto día del,anestesico local,1,[CLS] anestesico local [SEP] ( EVA < 2 ) con la nueva concentración de < start > AL < end > El catéter epidural se retiró en el quinto día del [SEP],0


Acrónimos que falla

In [96]:
wrong_test.groupby('short_form').size().reset_index().sort_values(0, ascending = False)

,short_form,0
59,mg,385
4,AV,76
60,mmol,27
7,C3,19
57,c,18
...,...,...
8,C4,2
6,C.,2
16,CHOP,2
47,Q,1


Estudiamos si aquellos acrónimos que no acierta no tenían muchos registros en train

In [168]:
wrong_lf = wrong_test['long_form'].unique().tolist()

In [169]:
len(wrong_lf)

166

In [170]:
train.shape

(7297, 5)

In [171]:
len(train['long_form'].unique().tolist())

735

In [172]:
group_lf = train.groupby('long_form').size().reset_index().sort_values(0)

In [173]:
lf_join = group_lf[group_lf['long_form'].isin(wrong_lf)]
lf_join.shape

(44, 2)

In [174]:
lf_join

,long_form,0
243,escherichia,1
690,tromboplastina parcial activado,1
516,onda p onda r,1
502,neutrofilo,1
72,aspartic acid,1
111,carbohydrate antigen 199,1
136,cluster diferenciacion 117,1
673,tiempo relajacion longitudinal,2
370,infarto agudo miocardio,2
268,figura,2


Veamos cuáles hay en test que no están en train

In [175]:
lf_notrain = set(wrong_lf) ^ set(lf_join['long_form'].unique().tolist())
len(lf_notrain)

122

In [114]:
lf_notrain

{'adenina',
 'adeniner',
 'agudeza visual',
 'alanine aminotransferar',
 'alanine transferar',
 'aldosterona plasmatico',
 'amiloidosis primario',
 'analisis inmunoabsorcion ligado enzima',
 'anestesico local',
 'arteriovenoso',
 'aspartic acid',
 'atencion primario',
 'auriculoventricular',
 'bronchoalveolar lavage',
 'california',
 'cancer antiger',
 'cancer antiger 199',
 'carbohydrate antigen 199',
 'carbohydrate antiger',
 'chlamydia',
 'chloryde',
 'ciclofosfamido hidroxidaunomicina oncovin prednisono',
 'citrobacter',
 'cluster diferenciacion 117',
 'cluster of differentiation 117',
 'cluster of differentiation 34',
 'cluster of differentiation 8',
 'coding dno',
 'coding dno sequence',
 'coeficiente intelectual',
 'coeficiente inteligencia',
 'complemento 4',
 'comprimido',
 'creatinar kinar',
 'cuarto derivacion precordial',
 'cumulo diferenciacion 117',
 'cumulo diferenciacion 34',
 'cyclophosphamidir hydroxydaunomycin oncovin prednisoner',
 'derecho',
 'derivacion precordial

De las 126 definiciones que hay en test, sólo 12 están en train. Puede ser porque elimino aquellas definiciones no ambiguas.

In [117]:
train_all = pd.read_csv("data/data_train/train_data_beto_10.csv")

In [123]:
train_all.shape

(6360, 5)

In [118]:
len(train_all['long_form'].unique().tolist())

767

In [120]:
group_all = train_all.groupby('long_form').size().reset_index().sort_values(0)

In [121]:
lf_join_all = group_all[group_all['long_form'].isin(wrong_lf)]
lf_join_all.shape

(56, 2)

In [122]:
print(f"El número de definiciones que no acierta en test es {len(wrong_lf)}")
print(f"El número de definiciones de estas que están en todo train es {len(lf_join_all)}")

El número de definiciones que no acierta en test es 126
El número de definiciones de estas que están en todo train es 56


#### Estudiemos cada acrónimo que da fallo

In [176]:
#Lista de definiciones que no acierta en test
wrong_lf

['cancer antiger',
 'faceta articular',
 'miligramar',
 'filamentous',
 'fenilalanina',
 'french',
 'milimetro mercurio',
 'emision positron',
 'tamano 2',
 'primero vertebra toracico',
 'arteriovenoso',
 'auriculoventricular',
 'fibroma osificante',
 'movimiento mano',
 'mieloma multiple',
 'klebsiella',
 'fosfatasa alcalino',
 'vida medio',
 'microgramo',
 'picogramo',
 'human leucocyte antiger',
 'tercero componente complemento',
 'complemento 3',
 'nivel c subnivel 3',
 'proteina c reactivo',
 'polimerase chain reaction',
 'cluster of differentiation 117',
 'milimol',
 'enzymelinked immunosorbent assay',
 'analisis inmunoabsorcion ligado enzima',
 'chloryde',
 'velocidad eritrosedimentacion',
 'nanogramar',
 'american joint committee cancer',
 'centimetro',
 'kalium',
 'chlamydia',
 'escherichia',
 'millon unidades',
 'anatomia patologico',
 'tyrosine kinar',
 'cuarto componente complemento',
 'complemento 4',
 'tercero vertebra cervical',
 'litro',
 'tuberculosis',
 'listeria',
 '

Lo buscamos en todo train y en test

In [180]:
train[train['long_form'] == 'faceta articular'].shape

(0, 5)

In [184]:
test[test['long_form'] == 'faceta articular']

,short_form,context,long_form,label,sentences,Prediction
23,FA,"UI/ml , GPT 371 UI/ml , GGT 632 UI/ml , < start > FA < end > 787 UI/ml , LDH 394 UI/ml . Amilasa y lipasa",faceta articular,0,"[CLS] faceta articular [SEP] UI/ml , GPT 371 UI/ml , GGT 632 UI/ml , < start > FA < end > 787 UI/ml , LDH 394 UI/ml . Amilasa y lipasa [SEP]",1
298,FA,"UI/l , lactatodeshidrogenasa ( LDH ) 254 UI/l , FA < start > 1 < end > 7 UI/l . Iones : Na 130 mEq/l , K",faceta articular,0,"[CLS] faceta articular [SEP] UI/l , lactatodeshidrogenasa ( LDH ) 254 UI/l , FA < start > 1 < end > 7 UI/l . Iones : Na 130 mEq/l , K [SEP]",1
1237,FA,"ambas articulares , curetaje de las carillas de las FA < start > s < end > periores de T12 , y reducción abierta de la luxación",faceta articular,1,"[CLS] faceta articular [SEP] ambas articulares , curetaje de las carillas de las FA < start > s < end > periores de T12 , y reducción abierta de la luxación [SEP]",1


Buscamos el acrónimo correspondiente a esa definición en train, para estudiar que puede estar pasando

In [182]:
train[train['short_form'] == 'FA']['long_form'].unique()

array(['fluorescein angiography', 'fosfatasa alcalina',
       'autofluorescencia fondo', 'fibrilacion auricular'], dtype=object)

In [159]:
train_all[train_all['short_form'] == 'AV']['long_form'].unique()

array(['agudeza visual'], dtype=object)

In [188]:
for i in wrong_lf:
    train_rows = train[train['long_form'] == i].shape[0]
    sf = test[test['long_form'] == i]['short_form'].iloc[0]
    lfs = train[train['short_form'] == sf]['long_form'].unique()
    print(f"Para la lf {i} hay {train_rows} filas en train")
    print(f"Las definiciones de {sf} en trrain son {lfs}")

Para la lf cancer antiger hay 0 filas en train
Las definiciones de CA en trrain son ['cancer antigen' 'camara anterior']
Para la lf faceta articular hay 0 filas en train
Las definiciones de FA en trrain son ['fluorescein angiography' 'fosfatasa alcalina' 'autofluorescencia fondo'
 'fibrilacion auricular']
Para la lf miligramar hay 0 filas en train
Las definiciones de mg en trrain son ['miligramo']
Para la lf filamentous hay 0 filas en train
Las definiciones de F en trrain son ['french']
Para la lf fenilalanina hay 0 filas en train
Las definiciones de F en trrain son ['french']
Para la lf french hay 2 filas en train
Las definiciones de F en trrain son ['french']
Para la lf milimetro mercurio hay 213 filas en train
Las definiciones de mmHg en trrain son ['milimetro mercurio']
Para la lf emision positron hay 0 filas en train
Las definiciones de PET en trrain son ['positron emission tomography']
Para la lf tamano 2 hay 0 filas en train
Las definiciones de T2 en trrain son ['tiempo relajaci

### Study metrics values

In [194]:
test[(test['label'] == 1) & (test['Prediction'] == 1)].shape

(3367, 6)

In [195]:
test.shape

(4419, 6)

In [30]:
acr = 'PCR'

text = 'Nuria y Poveda se han hecho una PCR por la fiesta del sábado'

In [31]:
start_i = text.find(acr)
end_i = start_i + len(acr)

In [32]:
text[start_i:end_i]

'PCR'